In [ ]:
!git clone https://github.com/pallets/flask.git

In [ ]:
# %pip install networkx arango langgraph langchain_openai langchain_community matplotlib scipy

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import os
import ast
from typing import Dict, Set, List, Tuple, Optional,Any
import json
from arango import ArangoClient
import nx_arangodb as nxadb
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool
from langchain_mistralai import ChatMistralAI
import re

### Building graph

In [ ]:
# import os
# import ast
# import networkx as nx
# from typing import Dict, Set, List
# import json
# from arango import ArangoClient
# os.environ["MISTRAL_API_KEY"]="jJAuJZkjVcy2ynUhan375sHNviHiBeJU"

# class CodebaseVisualizer:
#     def __init__(self, root_dir: str):
#         self.root_dir = root_dir
#         self.graph = nx.DiGraph()
#         self.file_contents: Dict[str, str] = {}
#         self.import_relations: Dict[str, Set[str]] = {}
#         self.module_symbols: Dict[str, Dict[str, Dict[str, int]]] = {}  # file -> {symbol -> {type, line_no}}
#         self.file_index: Dict[str, int] = {}  # Maps files to indices
#         self.current_index = 0
#         self.directories: Set[str] = set()

#     def _get_next_index(self) -> int:
#         """Get next available index for file indexing."""
#         self.current_index += 1
#         return self.current_index

#     def _chunk_code(self, code: str, lines_per_chunk: int = 20) -> List[Dict]:
#         """
#         Chunk the given code into snippets.
#         Returns a list of dictionaries with 'code_snippet', 'start_line', and 'end_line'.
#         """
#         lines = code.splitlines()
#         chunks = []
#         for i in range(0, len(lines), lines_per_chunk):
#             chunk_lines = lines[i:i + lines_per_chunk]
#             chunk = {
#                 'code_snippet': '\n'.join(chunk_lines),
#                 'start_line': i + 1,
#                 'end_line': i + len(chunk_lines)
#             }
#             chunks.append(chunk)
#         return chunks

#     def parse_files(self) -> None:
#         """Parse all Python files in the directory and build relationships."""
#         # First pass: Index all files and create directory nodes
#         for root, dirs, files in os.walk(self.root_dir):
#             # Add directory node
#             rel_dir = os.path.relpath(root, self.root_dir)
#             if rel_dir != '.':
#                 self.directories.add(rel_dir)
#                 self.graph.add_node(rel_dir, type='directory')

#             # Index Python files
#             for file in files:
#                 if file.endswith('.py'):
#                     file_path = os.path.join(root, file)
#                     rel_path = os.path.relpath(file_path, self.root_dir)
#                     self.file_index[rel_path] = self._get_next_index()
                    
#                     try:
#                         with open(file_path, 'r', encoding='utf-8') as f:
#                             content = f.read()
#                             self.file_contents[rel_path] = content
#                             self._analyze_file(rel_path, content)
#                     except Exception as e:
#                         print(f"Error parsing {file_path}: {e}")

#     def _analyze_file(self, file_path: str, content: str) -> None:
#         """Analyze a single file for imports and symbols with line numbers."""
#         try:
#             tree = ast.parse(content)
#             imports = set()
#             symbols = {}

#             for node in ast.walk(tree):
#                 # Track imports
#                 if isinstance(node, (ast.Import, ast.ImportFrom)):
#                     if isinstance(node, ast.Import):
#                         for name in node.names:
#                             imports.add((name.name, node.lineno))
#                     else:  # ImportFrom
#                         module = node.module if node.module else ''
#                         imports.add((module, node.lineno))

#                 # Track defined symbols with line numbers
#                 elif isinstance(node, (ast.FunctionDef, ast.ClassDef)):
#                     symbols[node.name] = {
#                         'type': 'class' if isinstance(node, ast.ClassDef) else 'function',
#                         'line_no': node.lineno
#                     }

#             self.import_relations[file_path] = imports
#             self.module_symbols[file_path] = symbols

#         except Exception as e:
#             print(f"Error analyzing {file_path}: {e}")

#     def build_graph(self) -> nx.DiGraph:
#         """Build the NetworkX graph with enhanced node and edge information."""
#         # Start with a directed graph for clarity in relationships
#         dot_graph = nx.DiGraph()
        
#         # Add nodes for all files with indices and code snippet nodes
#         for file_path, file_idx in self.file_index.items():
#             dot_graph.add_node(file_path, 
#                                type='file',
#                                file_index=file_idx,
#                                directory=os.path.dirname(file_path))
            
#             # Create snippet nodes for the entire file
#             if file_path in self.file_contents:
#                 chunks = self._chunk_code(self.file_contents[file_path])
#                 for idx, chunk_info in enumerate(chunks):
#                     snippet_node = f"{file_path}::snippet::{idx}"
#                     dot_graph.add_node(snippet_node,
#                                        type='snippet',
#                                        code_snippet=chunk_info['code_snippet'],
#                                        start_line=chunk_info['start_line'],
#                                        end_line=chunk_info['end_line'])
#                     # Connect file node to snippet node
#                     dot_graph.add_edge(file_path, snippet_node, 
#                                        edge_type='contains_snippet',
#                                        start_line=chunk_info['start_line'],
#                                        end_line=chunk_info['end_line'])

#             # Add nodes for symbols in this file
#             for symbol, details in self.module_symbols.get(file_path, {}).items():
#                 symbol_node = f"{file_path}::{symbol}"
#                 dot_graph.add_node(symbol_node, 
#                                    type='symbol',
#                                    symbol_type=details['type'],
#                                    line_number=details['line_no'])
#                 dot_graph.add_edge(file_path, symbol_node, 
#                                    edge_type='defines',
#                                    line_number=details['line_no'])

#         # Add directory nodes
#         for directory in self.directories:
#             dot_graph.add_node(directory, type='directory')

#         # Add edges for imports with line numbers
#         for file_path, imports in self.import_relations.items():
#             for imp, line_no in imports:
#                 # Look for matching files or symbols
#                 for target_file, symbols in self.module_symbols.items():
#                     if imp in symbols:
#                         dot_graph.add_edge(file_path, 
#                                            f"{target_file}::{imp}",
#                                            edge_type='import',
#                                            line_number=line_no)
#                     elif target_file.replace('.py', '').endswith(imp):
#                         dot_graph.add_edge(file_path, 
#                                            target_file,
#                                            edge_type='import',
#                                            line_number=line_no)
        
#         # Save the built graph in self.graph for later export (to ArangoDB, JSON, etc.)
#         self.graph = dot_graph
#         return dot_graph

#     def export_file_index(self, output_path: str = 'file_index.txt') -> None:
#         """Export the file index mapping to a text file."""
#         with open(output_path, 'w') as f:
#             # Write header
#             f.write("File Index Mapping\n")
#             f.write("=================\n\n")
            
#             # Sort by index for better readability
#             sorted_items = sorted(self.file_index.items(), key=lambda x: x[1])
            
#             # Write each file and its index
#             for file_path, index in sorted_items:
#                 f.write(f"{index}: {file_path}\n")
                
#                 # If there are symbols in this file, list them with line numbers
#                 if file_path in self.module_symbols:
#                     f.write("  Symbols:\n")
#                     for symbol, details in self.module_symbols[file_path].items():
#                         symbol_type = details['type']
#                         line_no = details['line_no']
#                         f.write(f"    - {symbol} ({symbol_type}) at line {line_no}\n")
#                     f.write("\n")

#     def export_graph_json(self, output_path: str = 'codebase_graph.json') -> None:
#         """Export the graph structure to JSON."""
#         graph_data = {
#             'nodes': [
#                 {
#                     'id': node,
#                     'type': data['type'],
#                     'file_index': data.get('file_index'),
#                     'directory': data.get('directory'),
#                     'symbol_type': data.get('symbol_type'),
#                     'line_number': data.get('line_number'),
#                     'code_snippet': data.get('code_snippet'),
#                     'start_line': data.get('start_line'),
#                     'end_line': data.get('end_line')
#                 } 
#                 for node, data in self.graph.nodes(data=True)
#             ],
#             'links': [
#                 {
#                     'source': source,
#                     'target': target,
#                     'type': data.get('edge_type'),
#                     'line_number': data.get('line_number'),
#                     'start_line': data.get('start_line'),
#                     'end_line': data.get('end_line')
#                 } 
#                 for source, target, data in self.graph.edges(data=True)
#             ]
#         }
        
#         with open(output_path, 'w') as f:
#             json.dump(graph_data, f, indent=2)
#     def export_to_arango(self,
#                          db_name: str = 'codebase',
#                          username: str = 'root',
#                          password: str = 'passwd',
#                          host: str = 'http://localhost:8529') -> None:
#         """
#         Export the graph into ArangoDB.
#         IMPORTANT: This method first deletes any existing graph and associated data in ArangoDB.
#         """
#         client = ArangoClient(hosts=host)
#         db = client.db(username=username, password=password,verify=True)

#         # Delete the existing graph and its collections if they exist.
#         graph_name = "FlaskRepv1"
#         '''
#         if db.has_graph(graph_name):
#             try:
#                 # Delete the entire graph and its collections.
#                 db.graph(graph_name).delete(delete_collections=True)
#                 print("Existing ArangoDB graph deleted.")
#             except Exception as e:
#                 print(f"Error deleting graph: {e}")
#                 '''
        
#         G_adb = nxadb.Graph(
#             name=graph_name,
#             db=db,
#             incoming_graph_data=self.graph,
#             write_batch_size=50000,
#             overwrite_graph=True
#         )
        
#         self.G_adb=G_adb
#         return G_adb
#         print("Graph successfully exported to ArangoDB.")
#     def text_to_nx_algorithm_to_text(self,query):
#         """This tool is available to invoke a NetworkX Algorithm on
#         the ArangoDB Graph. You are responsible for accepting the
#         Natural Language Query, establishing which algorithm needs to
#         be executed, executing the algorithm, and translating the results back
#         to Natural Language, with respect to the original query.
    
#         If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
#         this tool.
#         """
#         llm = ChatMistralAI(
#             model="mistral-large-latest",
#             temperature=0,
#             max_retries=2,
#             # other params...
#         )
#         ######################
#         print("1) Generating NetworkX code")
    
#         text_to_nx = llm.invoke(f"""
#         I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}
    
#         I have the following graph analysis query: {query}.
    
#         Generate the Python Code required to answer the query using the `G_adb` object.
        
#         It should give code so that is gives ALL the necessary contents that use this part of the code in terms of ALL nested imports. Consider the nested directories and sub-directory informations as well.
    
#         Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.
    
#         Only assume that networkx is installed, and other base python dependencies.
    
#         Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.
    
#         Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.
    
#         Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.
    
#         Your code:
#         """).content
    
#         text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
        
#         print('-'*10)
#         print(text_to_nx_cleaned)
#         print('-'*10)
    
#         ######################
    
#         print("\n2) Executing NetworkX code")
#         global_vars = {"G_adb":self.G_adb , "nx": nx}
#         local_vars = {}
    
#         try:
#             exec(text_to_nx_cleaned, global_vars, local_vars)
#             text_to_nx_final = text_to_nx
#         except Exception as e:
#             print(f"EXEC ERROR: {e}")
#             return f"EXEC ERROR: {e}"
    
#             # TODO: Consider experimenting with a code corrector!
#             attempt = 1
#             MAX_ATTEMPTS = 3
    
#             # while attempt <= MAX_ATTEMPTS
#                 # ...
    
#         print('-'*10)
#         FINAL_RESULT = local_vars["FINAL_RESULT"]
#         print(f"FINAL_RESULT: {FINAL_RESULT}")
#         print('-'*10)
    
#         ######################
    
#         print("3) Formulating final answer")
    
#         nx_to_text = llm.invoke(f"""
#             I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}
    
#             I have the following graph analysis query: {query}.
    
#             I have executed the following python code to help me answer my query:
    
#             ---
#             {text_to_nx_final}
#             ---
    
#             The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.
    
#             Based on my original Query and FINAL_RESULT, generate a short and concise response to
#             answer my query.
            
#             Your response:
#         """).content
    
#         return nx_to_text

# # Example usage

# # Initialize and parse the codebase
# visualizer = CodebaseVisualizer("flask")
# visualizer.parse_files()
# visualizer.build_graph()
    
# # Export the file index and JSON (for local inspection)
# visualizer.export_file_index()
# visualizer.export_graph_json()
# # Export the enriched graph to ArangoDB (this will delete any existing graph data first)
# #visualizer.export_to_arango(db_name="FlaskRepv1",username="root",password="cUZ0YaNdcwfUTw6VjRny",host="http://localhost:8529")


In [ ]:
#visualizer.export_to_arango(db_name="FlaskRepv1",username="root",password="cUZ0YaNdcwfUTw6VjRny",host="https://d2eeb8083350.arangodb.cloud:8529")


In [ ]:
visualizer = CodebaseVisualizer("flask")
visualizer.parse_files()
G = visualizer.build_graph()

In [ ]:
class GraphVisualizer:
    def __init__(self, graph: nx.Graph):
        self.graph = graph
        self.pos = None
        
    def set_layout(self, layout_type: str = 'spring', **layout_params) -> None:
        """
        Set the layout for the graph visualization.
        
        Args:
            layout_type: Type of layout ('spring', 'circular', 'kamada_kawai', 
                        'random', 'shell', 'spectral')
            layout_params: Additional parameters for the layout algorithm
        """
        layout_funcs = {
            'spring': nx.spring_layout,
            'circular': nx.circular_layout,
            'kamada_kawai': nx.kamada_kawai_layout,
            'random': nx.random_layout,
            'shell': nx.shell_layout,
            'spectral': nx.spectral_layout
        }
        
        if layout_type not in layout_funcs:
            raise ValueError(f"Unsupported layout type. Choose from: {list(layout_funcs.keys())}")
            
        self.pos = layout_funcs[layout_type](self.graph, **layout_params)
    
    def _get_node_colors(self) -> Dict[str, str]:
        """Extract node colors from graph attributes or generate defaults."""
        colors = {}
        for node in self.graph.nodes():
            # Check for color in node attributes
            attrs = self.graph.nodes[node]
            if 'fillcolor' in attrs:
                colors[node] = attrs['fillcolor']
            elif 'color' in attrs:
                colors[node] = attrs['color']
            else:
                colors[node] = 'lightblue'  # default color
        return colors
    
    def _get_node_sizes(self) -> Dict[str, float]:
        """Extract or compute node sizes."""
        sizes = {}
        for node in self.graph.nodes():
            attrs = self.graph.nodes[node]
            if 'size' in attrs:
                sizes[node] = attrs['size']
            else:
                # Default size based on node degree
                sizes[node] = 1000 * (1 + self.graph.degree(node) / 10)
        return sizes
    
    def _get_edge_colors(self) -> Dict[Tuple[str, str], str]:
        """Extract edge colors from graph attributes or generate defaults."""
        colors = {}
        for u, v in self.graph.edges():
            edge_data = self.graph.get_edge_data(u, v)
            if 'color' in edge_data:
                colors[(u, v)] = edge_data['color']
            else:
                colors[(u, v)] = 'gray'  # default color
        return colors
    
    def _get_node_labels(self) -> Dict[str, str]:
        """Extract node labels from graph attributes."""
        labels = {}
        for node in self.graph.nodes():
            attrs = self.graph.nodes[node]
            if 'label' in attrs:
                labels[node] = attrs['label']
            else:
                labels[node] = str(node)
        return labels
    
    def _get_edge_labels(self) -> Dict[Tuple[str, str], str]:
        """Extract edge labels from graph attributes."""
        labels = {}
        for u, v in self.graph.edges():
            edge_data = self.graph.get_edge_data(u, v)
            if 'label' in edge_data:
                labels[(u, v)] = edge_data['label']
        return labels

    def visualize(self, 
                 figsize: Tuple[int, int] = (12, 8),
                 node_size: Optional[Dict[str, float]] = None,
                 node_color: Optional[Dict[str, str]] = None,
                 edge_color: Optional[Dict[Tuple[str, str], str]] = None,
                 with_labels: bool = True,
                 font_size: int = 8,
                 title: Optional[str] = None,
                 show_edge_labels: bool = True,
                 alpha: float = 0.7,
                 save_path: Optional[str] = None) -> None:
        """
        Visualize the graph with customizable options.
        
        Args:
            figsize: Size of the figure (width, height)
            node_size: Dictionary mapping nodes to their sizes
            node_color: Dictionary mapping nodes to their colors
            edge_color: Dictionary mapping edges to their colors
            with_labels: Whether to show node labels
            font_size: Size of the font for labels
            title: Title of the graph
            show_edge_labels: Whether to show edge labels
            alpha: Transparency of nodes
            save_path: Path to save the visualization (if None, displays instead)
        """
        if self.pos is None:
            self.set_layout('spring')
            
        plt.figure(figsize=figsize)
        
        # Get or use provided node attributes
        node_colors = node_color if node_color is not None else self._get_node_colors()
        node_sizes = node_size if node_size is not None else self._get_node_sizes()
        edge_colors = edge_color if edge_color is not None else self._get_edge_colors()
        
        # Draw nodes
        nx.draw_networkx_nodes(self.graph, self.pos,
                             node_color=[node_colors[node] for node in self.graph.nodes()],
                             node_size=[node_sizes[node] for node in self.graph.nodes()],
                             alpha=alpha)
        
        # Draw edges
        for (u, v) in self.graph.edges():
            nx.draw_networkx_edges(self.graph, self.pos,
                                 edgelist=[(u, v)],
                                 edge_color=edge_colors.get((u, v), 'gray'),
                                 alpha=0.5)
        
        # Add labels if requested
        if with_labels:
            labels = self._get_node_labels()
            nx.draw_networkx_labels(self.graph, self.pos, labels,
                                  font_size=font_size)
        
        # Add edge labels if requested
        if show_edge_labels:
            edge_labels = self._get_edge_labels()
            if edge_labels:
                nx.draw_networkx_edge_labels(self.graph, self.pos,
                                           edge_labels=edge_labels,
                                           font_size=font_size-2)
        
        if title:
            plt.title(title)
        
        plt.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
            plt.close()
        else:
            plt.show()

# Example usage:
'''
# Create a sample graph
G = nx.Graph()
G.add_nodes_from([
    (1, {'fillcolor': 'lightblue', 'label': 'Node 1'}),
    (2, {'fillcolor': 'lightgreen', 'label': 'Node 2'}),
    (3, {'fillcolor': 'lightred', 'label': 'Node 3'})
])
G.add_edges_from([
    (1, 2, {'color': 'blue', 'label': 'Edge 1-2'}),
    (2, 3, {'color': 'red', 'label': 'Edge 2-3'})
])
'''
# Create visualizer and display graph
visualizer = GraphVisualizer(G)
visualizer.set_layout('spring', k=2)  # k controls the spacing between nodes
visualizer.visualize(
    figsize=(10, 8),
    font_size=10,
    title="Sample Graph Visualization",
)


### Make Database


In [ ]:
'''
db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True)

print(db)
'''

In [ ]:
'''
G_adb = nxadb.Graph(
    name="FlaskRepv1",
    db=db,
    incoming_graph_data=G,
    write_batch_size=50000 # feel free to modify
)

print(G_adb)
'''

### Run from loaded database

In [ ]:
db = ArangoClient(hosts="https://d2eeb8083350.arangodb.cloud:8529").db(username="root", password="cUZ0YaNdcwfUTw6VjRny", verify=True)

print(db)

In [ ]:
G_adb = nxadb.Graph(
    name="FlaskRepv1",
    db=db,
    #incoming_graph_data=G,
    #write_batch_size=50000 # feel free to modify
)

print(G_adb)

In [ ]:
arango_graph = ArangoGraph(db)

In [ ]:
os.environ["MISTRAL_API_KEY"]="jJAuJZkjVcy2ynUhan375sHNviHiBeJU"

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    # other params...
)
llm.invoke("Hello")

In [ ]:

@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """
    llm = ChatMistralAI(
        model="mistral-large-latest",
        temperature=0,
        max_retries=2,
        # other params...
    )
    ######################
    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.
    
    It should give code so that is gives ALL the necessary contents that use this part of the code in terms of ALL nested imports. Consider the nested directories and sub-directory informations as well.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.

    Only assume that networkx is installed, and other base python dependencies.

    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & consice answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()
    
    print('-'*10)
    print(text_to_nx_cleaned)
    print('-'*10)

    ######################

    print("\n2) Executing NetworkX code")
    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        text_to_nx_final = text_to_nx
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"EXEC ERROR: {e}"

        # TODO: Consider experimenting with a code corrector!
        attempt = 1
        MAX_ATTEMPTS = 3

        # while attempt <= MAX_ATTEMPTS
            # ...

    print('-'*10)
    FINAL_RESULT = local_vars["FINAL_RESULT"]
    print(f"FINAL_RESULT: {FINAL_RESULT}")
    print('-'*10)

    ######################

    print("3) Formulating final answer")

    nx_to_text = llm.invoke(f"""
        I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

        I have the following graph analysis query: {query}.

        I have executed the following python code to help me answer my query:

        ---
        {text_to_nx_final}
        ---

        The `FINAL_RESULT` variable is set to the following: {FINAL_RESULT}.

        Based on my original Query and FINAL_RESULT, generate a short and concise response to
        answer my query.
        
        Your response:
    """).content

    return nx_to_text

In [ ]:
text_to_nx_algorithm_to_text("What is the use of has_level_handler in the codebase. Give appropriate context")

In [ ]:
print(G_adb)

In [ ]:
arango_graph.schema

In [ ]:
import os
import ast
import networkx as nx
from typing import Dict, Set, List, Tuple, Optional
import json
from arango import ArangoClient
import re

class CodebaseVisualizer:
    def __init__(self, root_dir: str):
        self.root_dir = root_dir
        self.graph = nx.DiGraph()
        self.file_contents: Dict[str, str] = {}
        self.import_relations: Dict[str, List[Tuple[str, int]]] = {}  # file -> [(module, line_no)]
        self.module_symbols: Dict[str, Dict[str, Dict[str, any]]] = {}  # file -> {symbol -> {type, line_no, context}}
        self.symbol_references: Dict[str, List[Tuple[str, int]]] = {}  # symbol -> [(file, line_no)]
        self.file_index: Dict[str, int] = {}  # Maps files to indices
        self.current_index = 0
        self.directories: Set[str] = set()

    def _get_next_index(self) -> int:
        """Get next available index for file indexing."""
        self.current_index += 1
        return self.current_index

    def _chunk_code(self, code: str, lines_per_chunk: int = 20) -> List[Dict]:
        """
        Chunk the given code into snippets.
        Returns a list of dictionaries with 'code_snippet', 'start_line', and 'end_line'.
        """
        lines = code.splitlines()
        chunks = []
        for i in range(0, len(lines), lines_per_chunk):
            chunk_lines = lines[i:i + lines_per_chunk]
            chunk = {
                'code_snippet': '\n'.join(chunk_lines),
                'start_line': i + 1,
                'end_line': i + len(chunk_lines)
            }
            chunks.append(chunk)
        return chunks

    def _get_context_around_line(self, file_path: str, line_no: int, context_lines: int = 3) -> str:
        """Extract context around a specific line in a file."""
        if file_path not in self.file_contents:
            return ""
        
        lines = self.file_contents[file_path].splitlines()
        start = max(0, line_no - context_lines - 1)
        end = min(len(lines), line_no + context_lines)
        
        context = "\n".join(lines[start:end])
        return context

    def parse_files(self) -> None:
        """Parse all Python files in the directory and build relationships."""
        # First pass: Index all files and create directory nodes
        for root, dirs, files in os.walk(self.root_dir):
            # Add directory node
            rel_dir = os.path.relpath(root, self.root_dir)
            if rel_dir != '.':
                self.directories.add(rel_dir)
                self.graph.add_node(rel_dir, type='directory')

            # Index Python files
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    rel_path = os.path.relpath(file_path, self.root_dir)
                    self.file_index[rel_path] = self._get_next_index()
                    
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                            self.file_contents[rel_path] = content
                            self._analyze_file(rel_path, content)
                    except Exception as e:
                        print(f"Error parsing {file_path}: {e}")
        
        # Second pass: Find symbol references across files
        self._find_symbol_references()

    def _analyze_file(self, file_path: str, content: str) -> None:
        """Analyze a single file for imports and symbols with line numbers and context."""
        try:
            tree = ast.parse(content)
            imports = []
            symbols = {}

            for node in ast.walk(tree):
                # Track imports
                if isinstance(node, (ast.Import, ast.ImportFrom)):
                    if isinstance(node, ast.Import):
                        for name in node.names:
                            imports.append((name.name, node.lineno))
                    else:  # ImportFrom
                        module = node.module if node.module else ''
                        for name in node.names:
                            imports.append((f"{module}.{name.name}" if module else name.name, node.lineno))

                # Track defined symbols with line numbers and context
                elif isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.Assign)):
                    if isinstance(node, (ast.FunctionDef, ast.ClassDef)):
                        symbol_name = node.name
                        symbol_type = 'class' if isinstance(node, ast.ClassDef) else 'function'
                        line_no = node.lineno
                        context = self._extract_node_source(content, node)
                        
                        symbols[symbol_name] = {
                            'type': symbol_type,
                            'line_no': line_no,
                            'context': context,
                            'docstring': ast.get_docstring(node)
                        }
                    elif isinstance(node, ast.Assign):
                        # Handle variable assignments
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                symbol_name = target.id
                                line_no = node.lineno
                                context = self._extract_node_source(content, node)
                                
                                symbols[symbol_name] = {
                                    'type': 'variable',
                                    'line_no': line_no,
                                    'context': context
                                }

            self.import_relations[file_path] = imports
            self.module_symbols[file_path] = symbols

        except Exception as e:
            print(f"Error analyzing {file_path}: {e}")

    def _extract_node_source(self, source: str, node) -> str:
        """Extract the source code for an AST node."""
        try:
            lines = source.splitlines()
            if hasattr(node, 'lineno') and hasattr(node, 'end_lineno'):
                start = node.lineno - 1
                end = getattr(node, 'end_lineno', start + 1)
                return '\n'.join(lines[start:end])
            return ""
        except Exception:
            return ""

    def _find_symbol_references(self) -> None:
        """Find references to symbols across all files."""
        for file_path, content in self.file_contents.items():
            try:
                tree = ast.parse(content)
                self._process_file_for_references(file_path, tree, content)
            except Exception as e:
                print(f"Error finding references in {file_path}: {e}")

    def _process_file_for_references(self, file_path: str, tree, source: str) -> None:
        """Process a file's AST to find references to symbols."""
        for node in ast.walk(tree):
            # Find variable references
            if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Load):
                symbol_name = node.id
                line_no = node.lineno
                
                # Track reference with context
                if symbol_name not in self.symbol_references:
                    self.symbol_references[symbol_name] = []
                
                context = self._get_context_around_line(file_path, line_no)
                self.symbol_references[symbol_name].append((file_path, line_no, context))
            
            # Find attribute references (e.g., obj.method())
            elif isinstance(node, ast.Attribute) and isinstance(node.ctx, ast.Load):
                attr_name = node.attr
                line_no = node.lineno
                
                if attr_name not in self.symbol_references:
                    self.symbol_references[attr_name] = []
                
                context = self._get_context_around_line(file_path, line_no)
                self.symbol_references[attr_name].append((file_path, line_no, context))

    def build_graph(self) -> nx.DiGraph:
        """Build the NetworkX graph with enhanced node and edge information."""
        # Start with a directed graph for clarity in relationships
        dot_graph = nx.DiGraph()
        
        # Add nodes for all files with indices and code snippet nodes
        for file_path, file_idx in self.file_index.items():
            dot_graph.add_node(file_path, 
                               type='file',
                               file_index=file_idx,
                               directory=os.path.dirname(file_path))
            
            # Create snippet nodes for the entire file
            if file_path in self.file_contents:
                chunks = self._chunk_code(self.file_contents[file_path])
                for idx, chunk_info in enumerate(chunks):
                    snippet_node = f"{file_path}::snippet::{idx}"
                    dot_graph.add_node(snippet_node,
                                       type='snippet',
                                       code_snippet=chunk_info['code_snippet'],
                                       start_line=chunk_info['start_line'],
                                       end_line=chunk_info['end_line'])
                    # Connect file node to snippet node
                    dot_graph.add_edge(file_path, snippet_node, 
                                       edge_type='contains_snippet',
                                       start_line=chunk_info['start_line'],
                                       end_line=chunk_info['end_line'])

            # Add nodes for symbols in this file
            for symbol, details in self.module_symbols.get(file_path, {}).items():
                symbol_node = f"{file_path}::{symbol}"
                dot_graph.add_node(symbol_node, 
                                   type='symbol',
                                   symbol_type=details['type'],
                                   line_number=details['line_no'],
                                   context=details.get('context', ''),
                                   docstring=details.get('docstring', ''))
                dot_graph.add_edge(file_path, symbol_node, 
                                   edge_type='defines',
                                   line_number=details['line_no'])

        # Add directory nodes
        for directory in self.directories:
            dot_graph.add_node(directory, type='directory')

        # Add edges for imports with line numbers
        for file_path, imports in self.import_relations.items():
            for imp, line_no in imports:
                # Look for matching files or symbols
                for target_file, symbols in self.module_symbols.items():
                    if imp in symbols:
                        dot_graph.add_edge(file_path, 
                                           f"{target_file}::{imp}",
                                           edge_type='import',
                                           line_number=line_no)
                    elif target_file.replace('.py', '').endswith(imp):
                        dot_graph.add_edge(file_path, 
                                           target_file,
                                           edge_type='import',
                                           line_number=line_no)
        
        # Add edges for symbol references with line numbers and context
        for symbol, references in self.symbol_references.items():
            for file_path, symbols in self.module_symbols.items():
                if symbol in symbols:
                    symbol_node = f"{file_path}::{symbol}"
                    
                    # Connect symbol to all its references
                    for ref_file, ref_line, context in references:
                        if ref_file != file_path:  # Only add cross-file references
                            dot_graph.add_edge(ref_file, 
                                              symbol_node,
                                              edge_type='references',
                                              line_number=ref_line,
                                              context=context)
        
        # Save the built graph in self.graph for later export
        self.graph = dot_graph
        return dot_graph

    def find_symbol_usages(self, symbol_name: str) -> List[Dict]:
        """Find all usages of a symbol in the codebase with context."""
        results = []
        
        # Look for the symbol in defined symbols
        for file_path, symbols in self.module_symbols.items():
            if symbol_name in symbols:
                details = symbols[symbol_name]
                results.append({
                    'file': file_path,
                    'type': 'definition',
                    'line': details['line_no'],
                    'symbol_type': details['type'],
                    'context': details.get('context', ''),
                    'docstring': details.get('docstring', '')
                })
        
        # Look for references to the symbol
        if symbol_name in self.symbol_references:
            for file_path, line_no, context in self.symbol_references[symbol_name]:
                results.append({
                    'file': file_path,
                    'type': 'reference',
                    'line': line_no,
                    'context': context
                })
        
        return results

    def export_file_index(self, output_path: str = 'file_index.txt') -> None:
        """Export the file index mapping to a text file."""
        with open(output_path, 'w') as f:
            # Write header
            f.write("File Index Mapping\n")
            f.write("=================\n\n")
            
            # Sort by index for better readability
            sorted_items = sorted(self.file_index.items(), key=lambda x: x[1])
            
            # Write each file and its index
            for file_path, index in sorted_items:
                f.write(f"{index}: {file_path}\n")
                
                # If there are symbols in this file, list them with line numbers
                if file_path in self.module_symbols:
                    f.write("  Symbols:\n")
                    for symbol, details in self.module_symbols[file_path].items():
                        symbol_type = details['type']
                        line_no = details['line_no']
                        f.write(f"    - {symbol} ({symbol_type}) at line {line_no}\n")
                    f.write("\n")

    def _analyze_symbol_purpose(self, symbol_name, usages):
        """Analyze the purpose of a symbol based on its usage patterns."""
        # Get the definition if available
        definitions = [u for u in usages if u['type'] == 'definition']
        references = [u for u in usages if u['type'] == 'reference']
        
        purpose = ""
        
        # Check if we have a definition with docstring
        if definitions and definitions[0].get('docstring'):
            purpose += f"{definitions[0]['docstring']}\n\n"
        
        # If it's a function, try to infer what it does from usage
        if definitions and definitions[0].get('symbol_type') == 'function':
            # Collect contexts where it's used
            contexts = [ref.get('context', '') for ref in references if ref.get('context')]
            
            # Analyze contexts for patterns
            if contexts:
                common_patterns = self._find_common_usage_patterns(contexts, symbol_name)
                
                purpose += "Based on usage patterns, this function appears to:\n"
                
                if any("assert" in ctx for ctx in contexts):
                    purpose += "- Be used in test assertions to verify behavior\n"
                
                if any("if" in ctx and symbol_name in ctx for ctx in contexts):
                    purpose += "- Be used as a condition in control flow statements\n"
                
                if common_patterns:
                    for pattern in common_patterns[:3]:  # Top 3 patterns
                        purpose += f"- {pattern}\n"
        
        # If we couldn't infer much, provide a generic description
        if not purpose or len(purpose.strip()) < 10:
            if definitions and definitions[0].get('symbol_type') == 'function':
                context = definitions[0].get('context', '')
                
                # Look for parameters to understand what it takes
                params = self._extract_function_params(context)
                
                purpose += f"This function appears to check or validate something"
                if params:
                    purpose += f" related to {', '.join(params)}"
                purpose += ".\n"
        
        return purpose

    def _find_common_usage_patterns(self, contexts, symbol_name):
        """Find common patterns in the usage contexts."""
        patterns = []
        
        # Check if it's used with certain objects/methods frequently
        if any(f".{symbol_name}" in ctx for ctx in contexts):
            patterns.append("Be a method called on objects")
        
        # Check if it's used for configuration or setup
        if any("config" in ctx.lower() or "setup" in ctx.lower() for ctx in contexts):
            patterns.append("Be involved in configuration or setup")
        
        # Check if it's used for logging
        if any("log" in ctx.lower() for ctx in contexts):
            patterns.append("Be related to logging functionality")
        
        # Check if it's used in exception handling
        if any("except" in ctx.lower() or "try" in ctx.lower() for ctx in contexts):
            patterns.append("Be involved in exception handling")
        
        return patterns

    def _extract_function_params(self, context):
        """Extract parameter names from a function definition."""
        params = []
        if context:
            # Simple regex-based extraction
            match = re.search(r'def\s+\w+\s*\((.*?)\)', context, re.DOTALL)
            if match:
                param_string = match.group(1)
                # Split by comma and clean up
                raw_params = [p.strip() for p in param_string.split(',')]
                # Extract just the parameter name (before any : or =)
                params = [re.split(r'[=:]', p)[0].strip() for p in raw_params if p]
                # Remove self if it's there
                if params and params[0] == 'self':
                    params = params[1:]
        return params

    def _general_codebase_analysis(self, query):
        """Provide a general analysis based on the query."""
        response = f"# Analysis for Query: {query}\n\n"
        
        # Check if the query is asking about structure
        if any(term in query.lower() for term in ['structure', 'organization', 'layout']):
            response += "## Codebase Structure\n\n"
            # Count files by directory
            files_by_dir = {}
            for file_path in self.file_index:
                directory = os.path.dirname(file_path)
                if directory not in files_by_dir:
                    files_by_dir[directory] = []
                files_by_dir[directory].append(file_path)
            
            response += f"The codebase contains {len(self.file_index)} Python files across {len(files_by_dir)} directories.\n\n"
            
            # Show top-level directories
            response += "Main directories:\n"
            for directory, files in sorted(files_by_dir.items(), key=lambda x: len(x[1]), reverse=True)[:10]:
                dir_name = directory if directory else '(root)'
                response += f"- {dir_name}: {len(files)} files\n"
        
        # Check if the query is asking about specific functionality
        functionality_terms = ['handle', 'process', 'create', 'generate', 'calculate']
        for term in functionality_terms:
            if term in query.lower():
                # Search for functions with this term
                matching_functions = []
                for file_path, symbols in self.module_symbols.items():
                    for symbol, details in symbols.items():
                        if details['type'] == 'function' and term in symbol.lower():
                            matching_functions.append((file_path, symbol, details))
                
                if matching_functions:
                    response += f"\n## Functions Related to '{term}'\n\n"
                    for file_path, symbol, details in matching_functions[:5]:  # Show top 5
                        response += f"- `{symbol}` in {file_path}:{details['line_no']}\n"
                    if len(matching_functions) > 5:
                        response += f"... and {len(matching_functions) - 5} more functions\n"
        
        return response

    def export_graph_json(self, output_path: str = 'codebase_graph.json') -> None:
        """Export the graph structure to JSON."""
        graph_data = {
            'nodes': [
                {
                    'id': node,
                    'type': data['type'],
                    'file_index': data.get('file_index'),
                    'directory': data.get('directory'),
                    'symbol_type': data.get('symbol_type'),
                    'line_number': data.get('line_number'),
                    'code_snippet': data.get('code_snippet'),
                    'start_line': data.get('start_line'),
                    'end_line': data.get('end_line'),
                    'context': data.get('context'),
                    'docstring': data.get('docstring')
                } 
                for node, data in self.graph.nodes(data=True)
            ],
            'links': [
                {
                    'source': source,
                    'target': target,
                    'type': data.get('edge_type'),
                    'line_number': data.get('line_number'),
                    'start_line': data.get('start_line'),
                    'end_line': data.get('end_line'),
                    'context': data.get('context')
                } 
                for source, target, data in self.graph.edges(data=True)
            ]
        }
        
        with open(output_path, 'w') as f:
            json.dump(graph_data, f, indent=2)

    def export_to_arango(self,
                         db_name: str = 'codebase',
                         username: str = 'root',
                         password: str = 'passwd',
                         host: str = 'http://localhost:8529') -> None:
        """Export the graph into ArangoDB."""
        client = ArangoClient(hosts=host)
        db = client.db(username=username, password=password, verify=True)

        # Delete the existing graph and its collections if they exist.
        graph_name = "FlaskRepv1"
        
        # Import networkx_to_arangodb if it's available
        try:
            import networkx_to_arangodb as nxadb # type: ignore
            G_adb = nxadb.Graph(
                name=graph_name,
                db=db,
                incoming_graph_data=self.graph,
                write_batch_size=50000,
                overwrite_graph=True
            )
            
            self.G_adb = G_adb
            print("Graph successfully exported to ArangoDB.")
            return G_adb
        except ImportError:
            print("networkx_to_arangodb module not found. Please install it to export to ArangoDB.")
            return None

    def find_variable_usages(self, variable_name: str) -> dict:
        """Find all usages of a variable in the codebase with detailed context."""
        results = {
            'definitions': [],
            'references': [],
            'summary': {}
        }
        
        # Find variable definitions
        for file_path, symbols in self.module_symbols.items():
            if variable_name in symbols:
                details = symbols[variable_name]
                if details['type'] == 'variable':
                    context_lines = self._get_context_around_line(file_path, details['line_no'])
                    results['definitions'].append({
                        'file': file_path,
                        'line': details['line_no'],
                        'context': context_lines
                    })
        
        # Find variable references
        if variable_name in self.symbol_references:
            for file_path, line_no, context in self.symbol_references[variable_name]:
                results['references'].append({
                    'file': file_path,
                    'line': line_no,
                    'context': context
                })
        
        # Generate summary
        results['summary'] = {
            'definition_count': len(results['definitions']),
            'reference_count': len(results['references']),
            'files_with_definitions': list(set(d['file'] for d in results['definitions'])),
            'files_with_references': list(set(r['file'] for r in results['references']))
        }
        
        return results

    def text_to_nx_algorithm_to_text(self, query):
        """
        Enhanced analysis function that uses the local graph to answer symbol queries
        without requiring external API calls.
        """
        # Check if this is a query about a symbol's usage
        symbol_pattern = r'(?:find usages? of|what is the use of) ([\w_]+)'
        symbol_match = re.search(symbol_pattern, query.lower())
        
        if symbol_match:
            symbol_name = symbol_match.group(1)
            usages = self.find_symbol_usages(symbol_name)
            
            if not usages:
                return f"No usages found for symbol '{symbol_name}' in the codebase."
            
            # Analyze the usage patterns
            definitions = [u for u in usages if u['type'] == 'definition']
            references = [u for u in usages if u['type'] == 'reference']
            
            # Organize by files
            files_with_usages = {}
            for usage in usages:
                file_path = usage['file']
                if file_path not in files_with_usages:
                    files_with_usages[file_path] = {'definitions': [], 'references': []}
                
                usage_type = usage['type']
                files_with_usages[file_path][usage_type + 's'].append(usage)
            
            # Start building response
            response = f"# Analysis of '{symbol_name}' in the Codebase\n\n"
            
            # Add definition section if available
            if definitions:
                definition = definitions[0]  # Get primary definition
                response += f"## Definition\n\n"
                symbol_type = definition.get('symbol_type', 'function')
                response += f"**Type:** {symbol_type}\n"
                response += f"**Defined in:** {definition['file']}:{definition['line']}\n\n"
                
                if definition.get('docstring'):
                    response += f"**Docstring:**\n```\n{definition['docstring']}\n```\n\n"
                
                if definition.get('context'):
                    response += f"**Implementation:**\n```python\n{definition['context']}\n```\n\n"
            
            # Analyze purpose based on usage patterns
            response += f"## Purpose and Usage\n\n"
            
            # Try to infer purpose based on context
            purpose_analysis = self._analyze_symbol_purpose(symbol_name, usages)
            response += purpose_analysis + "\n\n"
            
            # List usage examples
            response += f"## Usage Examples\n\n"
            
            # Show top 3-5 most representative usage examples
            example_count = min(5, len(references))
            for i in range(example_count):
                if i < len(references):
                    ref = references[i]
                    response += f"### Example {i+1}: {ref['file']}:{ref['line']}\n\n"
                    if ref.get('context'):
                        response += f"```python\n{ref['context']}\n```\n\n"
            
            # Add summary stats
            response += f"## Summary Statistics\n\n"
            response += f"- Total occurrences: {len(usages)}\n"
            response += f"- Definitions: {len(definitions)}\n"
            response += f"- References: {len(references)}\n"
            response += f"- Files containing this symbol: {len(files_with_usages)}\n"
            
            return response
        
        # Check if this is a query about a variable's usage
        var_pattern = r'find (?:where|how) ([\w_]+) is (?:used|referenced|defined)'
        var_match = re.search(var_pattern, query.lower())
        
        if var_match:
            var_name = var_match.group(1)
            usages = self.find_variable_usages(var_name)
            
            if not usages['definitions'] and not usages['references']:
                return f"No usages found for variable '{var_name}' in the codebase."
            
            # Format a nice response
            response = f"Analysis of variable '{var_name}':\n\n"
            
            if usages['definitions']:
                response += f"DEFINITIONS ({len(usages['definitions'])}):\n"
                for def_info in usages['definitions']:
                    response += f"In {def_info['file']}:{def_info['line']}\n"
                    if def_info.get('context'):
                        response += f"```python\n{def_info['context']}\n```\n"
                response += "\n"
            
            if usages['references']:
                response += f"REFERENCES ({len(usages['references'])}):\n"
                for ref_info in usages['references']:
                    response += f"In {ref_info['file']}:{ref_info['line']}\n"
                    if ref_info.get('context'):
                        response += f"```python\n{ref_info['context']}\n```\n"
            
            return response
        
        # Handle "what is" questions with more general analysis
        what_is_pattern = r'what is ([\w_]+)'
        what_is_match = re.search(what_is_pattern, query.lower())
        
        if what_is_match:
            item_name = what_is_match.group(1)
            
            # Try to find it as a symbol
            symbol_usages = self.find_symbol_usages(item_name)
            
            if symbol_usages:
                # It's a symbol, use the symbol analysis
                return self.text_to_nx_algorithm_to_text(f"find usages of {item_name}")
            
            # Try to find it as a file or directory
            file_matches = [f for f in self.file_index.keys() if item_name in f]
            dir_matches = [d for d in self.directories if item_name in d]
            
            if file_matches or dir_matches:
                response = f"# Analysis of '{item_name}' in the Codebase\n\n"
                
                if file_matches:
                    response += f"## Matching Files\n\n"
                    for file in file_matches[:10]:  # Limit to 10 files
                        response += f"- {file}\n"
                    if len(file_matches) > 10:
                        response += f"... and {len(file_matches) - 10} more files\n"
                    response += "\n"
                
                if dir_matches:
                    response += f"## Matching Directories\n\n"
                    for directory in dir_matches:
                        response += f"- {directory}\n"
                    response += "\n"
                
                return response
            
            return f"Could not find '{item_name}' as a symbol, file, or directory in the codebase."
        
        # For any other type of query, perform a more general analysis
        return self._general_codebase_analysis(query)

    def analyze_has_level_handler(self):
        """Specialized analysis for has_level_handler in the Flask codebase."""
        symbol_name = "has_level_handler"
        usages = self.find_symbol_usages(symbol_name)
        
        if not usages:
            return f"No usages found for '{symbol_name}' in the codebase."
        
        # Separate definitions and references
        definitions = [u for u in usages if u['type'] == 'definition']
        references = [u for u in usages if u['type'] == 'reference']
        
        # Build a comprehensive analysis
        response = f"# Analysis of 'has_level_handler' in Flask\n\n"
        
        # 1. Add definition and implementation
        if definitions:
            definition = definitions[0]
            response += "## Definition\n\n"
            response += f"Defined in: {definition['file']}:{definition['line']}\n"
            
            if definition.get('context'):
                response += "\n```python\n" + definition['context'] + "\n```\n\n"
        
        # 2. Explain purpose
        response += "## Purpose\n\n"
        response += "The `has_level_handler` function checks whether a logger has a handler that will handle "
        response += "messages of a certain level. It's primarily used to determine if a default handler needs "
        response += "to be added to a logger.\n\n"
        
        # 3. Analyze usage in logging.py
        logging_references = [r for r in references if 'logging.py' in r['file']]
        if logging_references:
            response += "## Usage in Flask's Logging System\n\n"
            response += "In Flask's logging module, `has_level_handler` is used to determine whether to add "
            response += "a default handler to a logger. If the logger doesn't have an appropriate handler, "
            response += "Flask adds one automatically.\n\n"
            
            for ref in logging_references:
                response += f"### In {ref['file']}:{ref['line']}\n\n"
                if ref.get('context'):
                    response += "```python\n" + ref['context'] + "\n```\n\n"
                response += "Here, Flask is checking if the logger needs a default handler and adding one if necessary.\n\n"
        
        # 4. Analyze usage in tests
        test_references = [r for r in references if 'test_' in r['file']]
        if test_references:
            response += "## Testing\n\n"
            response += f"The function is tested in {len(test_references)} locations within the test suite. "
            response += "These tests verify the function works correctly with different logger configurations:\n\n"
            
            test_scenarios = [
                "- Testing with a logger that has no handlers",
                "- Testing with a logger that has handlers at the appropriate level",
                "- Testing with a logger with propagation disabled",
                "- Testing with handlers at different logging levels"
            ]
            
            response += "\n".join(test_scenarios) + "\n\n"
            
            # Show a representative test
            if test_references:
                response += "### Example Test\n\n"
                ref = test_references[0]
                if ref.get('context'):
                    response += "```python\n" + ref['context'] + "\n```\n\n"
        
        # 5. Add a summary
        response += "## Summary\n\n"
        response += f"`has_level_handler` is a utility function used {len(references)} times in the Flask codebase. "
        response += "It's a critical component of Flask's logging system that ensures log messages are properly handled. "
        response += "The function prevents Flask from adding redundant handlers while ensuring that logs are captured "
        response += "at the appropriate level."
        
        return response  

In [39]:
result = visualizer.analyze_has_level_handler()
print(result)

AttributeError: 'CodebaseVisualizer' object has no attribute 'analyze_has_level_handler'